# Model Evaluation using PyTorch Processor

1. [Introduction](#Introduction)
2. [Prerequisites](#Prerequisites)
3. [Setup](#Setup)
4. [Dataset](#Dataset)
5. [Build a SageMaker Processing Job](#Build-a-SageMaker-Processing-Job)
    1. [Review Model Evaluation Script](#Model-Evaluation-Scripts)
    2. [Configure Processing Job](#Configure-Processing-Job)
6. [Review Outputs](#Review-Outputs)

## Prerequisites

Download the notebook into your environment, and you can run it by simply execute each cell in order. To understand what's happening, you'll need:

- Familiarity with Python and numpy
- Basic familiarity with AWS S3.
- Basic understanding of AWS Sagemaker.
- Basic familiarity with AWS Command Line Interface (CLI) -- ideally, you should have it set up with credentials to access the AWS account you're running this notebook from.
- SageMaker Studio is preferred for the full UI integration

## Setup

Setting up the environment, load the libraries, and define the parameter for the entire notebook.

Run the cell below to ensure latest version of SageMaker is installed in your kernel

In [4]:
!pip3 install gymnasium
!pip3 install xgboost
!pip3 install stable-baselines3
!pip3 install stable-baselines3[extra]
!pip3 install tqdm
!pip3 install rich

  Using cached gymnasium-1.0.0-py3-none-any.whl (958 kB)
  Using cached Farama_Notifications-0.0.4-py3-none-any.whl (2.5 kB)
  Using cached xgboost-2.1.2-py3-none-manylinux_2_28_x86_64.whl (153.9 MB)
  Using cached nvidia_nccl_cu12-2.23.4-py3-none-manylinux2014_x86_64.whl (199.0 MB)
  Using cached stable_baselines3-2.3.2-py3-none-any.whl (182 kB)
  Using cached gymnasium-0.29.1-py3-none-any.whl (953 kB)
  Attempting uninstall: gymnasium
    Found existing installation: gymnasium 1.0.0
    Uninstalling gymnasium-1.0.0:
      Successfully uninstalled gymnasium-1.0.0
  Using cached pygame-2.6.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (14.0 MB)
  Using cached tensorboard-2.18.0-py3-none-any.whl (5.5 MB)
  Using cached Shimmy-1.3.0-py3-none-any.whl (37 kB)
  Using cached AutoROM-0.6.1-py3-none-any.whl (9.4 kB)
  Using cached AutoROM.accept_rom_license-0.6.1-py3-none-any.whl
  Using cached ale_py-0.8.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.7 MB)
  Using

In [2]:
import sagemaker
from sagemaker import get_execution_role
import boto3

sagemaker_session = sagemaker.Session()
region = sagemaker_session.boto_region_name
account = sagemaker_session.account_id()
role = sagemaker.get_execution_role()

default_bucket = sagemaker_session.default_bucket()
base_job_prefix = "cloudd-rf"
s3_client = boto3.client("s3")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


## Build a SageMaker Processing Job
There are 3 types of processing jobs depending on which framework you want to use: [AWS Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/processing-job.html)

### Model Evaluation Scripts

In [ ]:
%%writefile ../code/rl_rfe_fusion.py
import argparse
import os
import argparse
import gymnasium as gym
import matplotlib.pyplot as plt
import numpy as np
import numpy.ma as ma
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
import xgboost as xgb
import os
import pickle

from tqdm import tqdm
from gymnasium import spaces
from matplotlib.pyplot import figure
from matplotlib.lines import Line2D
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from stable_baselines3 import PPO
from torch.utils.data import Dataset
from torch.utils.tensorboard.writer import SummaryWriter 

import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")

SIG_TYPES = [['2-ASK', ['ask', 2], 0],
             ['4-ASK', ['ask', 4], 1],
             ['8-ASK', ['ask', 8], 2],
             ['BPSK', ['psk', 2], 3],
             ['QPSK', ['psk', 4], 4],
             ['16-QAM', ['qam', 16], 5],
             ['Tone', ['constant'], 6],
             ['P-FMCW', ['p_fmcw'], 7]]
NUM_CLASSES = len(SIG_TYPES)
sig_names = [i[0] for i in SIG_TYPES]

NUM_TEAM_FEATURES = {
    1: 65,
    2: 512,
    3: 64,
    4: 256    
}

NUM_FEATURES = 16  # How many of the top features to select for feature reduction

def float_list(arg):
    return list(map(float, arg.split(',')))

def int_list(arg):
    return list(map(int, arg.split(',')))

def parse_args():
    parser = argparse.ArgumentParser()
    
    # Number of samples per file.
    parser.add_argument("--num-sensors", type=int, default=4)
    parser.add_argument("--input-path", type=str, default=os.getenv("SM_CHANNEL_VAL"))
    parser.add_argument("--output-path", type=str, default=os.getenv("SM_OUTPUT_DIR"))
    
    return parser.parse_known_args()

def load_data():
    data_path = os.path.join(args.input_path, 'fusion_data', 'combined_features_allsecond_to_last.npy')  # Expects file containing numpy array of shape (num_samples, num_features)
    labels_path = os.path.join(args.input_path, 'fusion_data', 'labels.npy')  # Expects file containing numpy array of shape (num_samples, num_classes)
    data = torch.from_numpy(np.load(data_path))
    labels = torch.from_numpy(np.load(labels_path))

    return data, labels

# These functions are from team 2's old code
def get_reward(train_data, train_labels, validation_data, validation_labels):
    clf = xgb.XGBClassifier(tree_method="hist", early_stopping_rounds=2, n_estimators=5)
    clf.fit(train_data, train_labels, eval_set=[(validation_data, validation_labels)], verbose=0)

    return np.sum(clf.predict(validation_data) == validation_labels) / validation_labels.shape[0]

def statistical_feature_selection(data, labels):
    rewards = {}
    curr_features = np.arange(data.shape[1])
    curr_data = data
    for n in [data.shape[1], 512, 256, 128, 64, 32, 16, 8, 4, 2, 1]:
    #for n in [1, 2, 4, 8, 16]:
        curr_data = curr_data[:, curr_features]
        train_data, validation_data, train_labels, validation_labels = train_test_split(np.array(curr_data, dtype=np.float32), np.array(labels, dtype=np.int32), test_size=0.2)

        selector = RFE(DecisionTreeClassifier(), n_features_to_select=n, step=100, verbose=1)
        selector = selector.fit(train_data, train_labels)

        clf = xgb.XGBClassifier(tree_method="hist", early_stopping_rounds=2)
        clf.fit(train_data[:,selector.support_], train_labels, eval_set=[(validation_data[:,selector.support_], validation_labels)], verbose=0)
        
        model_out = np.argmax(clf.predict(validation_data[:, selector.support_]), axis=1)
        num_correct = np.sum(model_out == np.argmax(validation_labels, axis=1))
        num_validation_labels = validation_labels.shape[0]
        acc = num_correct / num_validation_labels
        

        rewards[n] = acc
        curr_features = selector.support_

    sns.lineplot(rewards)
    plt.title("Accuracy of XGBoost vs number of features")
    plt.xlabel("Number of Features")
    plt.ylabel("Accuracy")

class FusionModel(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.num_classes = num_classes
        self.layers = nn.Sequential(
                nn.LazyLinear(1024),
                nn.ReLU(),
                nn.Linear(1024, 256),
                nn.ReLU(),
                nn.Linear(256, self.num_classes)
        )

    def forward(self, x):
        return self.layers(x)

class FusionDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]
    
class SensorFusionBestFeatureDatasetEnv(gym.Env):
    def __init__(self, data, labels, num_classes, n, save_clf=True, load_clf=False):
        self.data = np.array(data)
        self.labels = np.array(labels)
        self.n = n
        self.mask_value = -100000

        self.observation_space = spaces.MultiDiscrete([self.data.shape[1]] * self.n)
        self.action_space = spaces.Discrete(self.data.shape[1])

        train_data, validation_data, train_labels, validation_labels = train_test_split(self.data, self.labels, test_size=0.2)
        self.train_data = train_data
        self.train_labels = train_labels.astype(int)
        self.validation_data = validation_data
        self.validation_labels = validation_labels.astype(int)

        self.curr_reward = 0
        self.prev_reward = 0
        
        self.clf = None

    def extract_important_features_using_decision_tree(self):
        clf = RandomForestClassifier(n_estimators=100, verbose=1)
        clf.fit(self.train_data, self.train_labels)

        return clf.feature_importances_.argsort()[-self.n:]

    def mask_data(self, data, idxes):
        mask = np.ones(data.shape, dtype=bool)
        for i in range(mask.shape[0]):
            mask[i][idxes[i]] = 0
        data[mask] = self.mask_value
        return data

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.count = 0
        self.action_idxes = np.ones(self.n, dtype=int)

        return self.action_idxes, {}
    
    def step(self, action):
        if action in self.action_idxes:
            return self.action_idxes, 0, False, False, {}

        if self.count < self.n:
            self.action_idxes[self.count] = action
        else:
            self.action_idxes[np.random.randint(self.n)] = action

        accuracy = self.calculate_accuracy(self.action_idxes)

        self.count += 1
        if self.count == self.n:
            terminated = True
        else:
            terminated = False
    
        return self.action_idxes, accuracy, terminated, False, {}

    def calculate_accuracy(self, action_idxes):

        train_data = self.train_data[:, action_idxes]
        validation_data = self.validation_data[:, action_idxes]
    
        self.clf = xgb.XGBClassifier(tree_method="hist", early_stopping_rounds=2, n_estimators=5)
        self.clf.fit(train_data, self.train_labels, eval_set=[(validation_data, self.validation_labels)], verbose=0)

        model_out = np.argmax(self.clf.predict(validation_data), axis=0)
        num_correct = np.sum(model_out == np.argmax(self.validation_labels, axis=0))
        num_validation_labels = self.validation_labels.shape[0]
        acc = num_correct / num_validation_labels
        return acc

    def render(self):
        return

    def close(self):
        return

class FeatureSelectionModel(nn.Module):
    def __init__(self, n, num_features, env, epsilon=0.999, epsilon_decay=0.001):
        self.n = n
        self.num_features = num_features
        self.env = env
        self.epsilon = epsilon
        self.epsilon_decay = epsilon_decay
        self.episode = 0

        self.AOR = np.zeros(num_features)
        self.AOR_counts = np.zeros(num_features)

    def decay_epsilon(self):
        if self.episode > 1500:
            self.epsilon = max(self.epsilon - self.epsilon_decay, 0.0)
        elif self.episode > 500:
            self.epsilon = max(self.epsilon - self.epsilon_decay, 0.1)

    def update_AOR(self, action, reward):
        self.AOR[action] = (self.AOR[action] * self.AOR_counts[action] + reward) / (self.AOR_counts[action] + 1)
        self.AOR_counts[action] += 1

    def select_action(self, state, epsilon):
        if np.random.rand() < epsilon:
            return np.random.choice(np.arange(self.num_features))
        mask = np.zeros(self.AOR.shape)
        mask[state] = True
        return ma.array(np.copy(self.AOR), mask=mask).argmax()

    def learn(self, episodes=16):
        writer = SummaryWriter(f"runs/n_{self.n}_episodes_{episodes}")
        for ep in tqdm(range(episodes)):
            rewards = []
            terminated = False
            state, _ = self.env.reset()
            while not terminated:
                self.decay_epsilon()
    
                action = self.select_action(state, self.epsilon)
    
                state, reward, terminated, _, _ = env.step(action)
                rewards.append(reward)
                self.update_AOR(action, reward)
            writer.add_scalar('reward', rewards[-1], ep)
            writer.add_scalar('epsilon', self.epsilon, ep)
            self.episode += 1

def visualize_output_features(d, l, nc):
    feature_avgs = torch.stack([d[l.int()==i].mean(axis=0) for i in range(nc)])
    plt.figure(0, (15, 1))
    sns.heatmap(feature_avgs)
    plt.yticks(ticks=np.arange(nc) + 0.5, labels=['2-ASK', '4-ASK', '8-ASK', 'BPSK', 'QPSK', '16-QAM', 'Tone', 'P-FMCW'], rotation=0)
    plt.tick_params(axis='x', which='both', bottom=False, labelbottom=False)

def create_rl_fused_model(data, labels):
    global NUM_CLASSES
    global NUM_FEATURES

    def run_eval(num_episodes, policy):
        rewards = []
        for _ in tqdm(range(num_episodes)):
            obs, _ = env.reset()
            ep_rewards = []
            ep_actions = []
            terminated = False
            while not terminated:
                action = policy(obs)
                _, reward, terminated, _, _ = env.step(action)
                ep_rewards.append(reward)
                ep_actions.append(action)
            rewards.append(ep_rewards[-1])
        return np.mean(rewards)

    env = SensorFusionBestFeatureDatasetEnv(data, labels, NUM_CLASSES, NUM_FEATURES)
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    
    env.reset()
    model = PPO("MlpPolicy", env, verbose=1, batch_size=16, n_steps=64, device=device)
    model.learn(total_timesteps=512, progress_bar=False) #4096
    avg_model_rewards = run_eval(10, lambda obs: model.predict(obs)[0])
    print(f"Average rewards for RL agent: {avg_model_rewards}")
    print('RL Accuracy:', env.calculate_accuracy(env.action_idxes))

    # Save the trained model
    fusion_data_folder = os.path.join(args.output_path, 'fusion_data')
    os.makedirs(fusion_data_folder, exist_ok=True)
    env.clf.save_model(f'{fusion_data_folder}/rl_fused_2tl.json')

    # Save a list of the top contributing features we found
    with open(f'{fusion_data_folder}/rl_feature_idxes_2tl.pkl', 'wb') as f:
        pickle.dump(env.action_idxes, f)

def create_rfe_fused_model(data, labels):
    global NUM_FEATURES

    train_data, validation_data, train_labels, validation_labels = train_test_split(np.array(data, dtype=np.float32), np.array(labels, dtype=np.int32), test_size=0.2)
    selector = RFE(DecisionTreeClassifier(), n_features_to_select=NUM_FEATURES, step=100, verbose=1)
    selector = selector.fit(train_data, train_labels)
    clf = xgb.XGBClassifier(tree_method="hist", early_stopping_rounds=2)
    clf.fit(train_data[:,selector.support_], train_labels, eval_set=[(validation_data[:,selector.support_], validation_labels)], verbose=0)
    model_out = np.argmax(clf.predict(validation_data[:, selector.support_]), axis=0)
    num_correct = np.sum(model_out == np.argmax(validation_labels, axis=0))
    num_validation_labels = validation_labels.shape[0]
    acc = num_correct / num_validation_labels
    print('RFE Accuracy:', acc)

    # Save the trained model
    fusion_data_folder = os.path.join(args.output_path, 'fusion_data')
    os.makedirs(fusion_data_folder, exist_ok=True)
    clf.save_model(f'{fusion_data_folder}/rfe_fused_2tl.json')

    # Get the list of features selected using RFE
    rfe_selected_feats = [i for i, x in enumerate(selector.support_) if x]

    # Save the list of features
    with open(f'{fusion_data_folder}/rfe_feature_idxes_2tl.pkl', 'wb') as f:
        pickle.dump(rfe_selected_feats, f)

def get_model_accuracy(data, labels):
    accuracies_rfe = {}
    accuracies_rl = {}
    curr_features = np.arange(data.shape[1])
    curr_data = data
        
    num_feats_to_test = [data.shape[1], 512, 256, 128, 64, 32, 16, 8, 4, 2, 1]
        
    for n in num_feats_to_test:
        curr_data = curr_data[:, curr_features]
        train_data, validation_data, train_labels, validation_labels = train_test_split(np.array(curr_data, dtype=np.float32), np.array(labels, dtype=np.int32), test_size=0.2)
            
        selector = RFE(DecisionTreeClassifier(), n_features_to_select=n, step=100, verbose=1)
        selector = selector.fit(train_data, train_labels)
        clf = xgb.XGBClassifier(tree_method="hist", early_stopping_rounds=2)
        clf.fit(train_data[:,selector.support_], train_labels, eval_set=[(validation_data[:,selector.support_], validation_labels)], verbose=0)
        model_out = np.argmax(clf.predict(validation_data[:, selector.support_]), axis=0)
        num_correct = np.sum(model_out == np.argmax(validation_labels, axis=0))
        num_validation_labels = validation_labels.shape[0]
        acc = num_correct / num_validation_labels
        accuracies_rfe[n] = acc
        print('RFE:', acc)
        curr_features = selector.support_
            
        env = SensorFusionBestFeatureDatasetEnv(data, labels, NUM_CLASSES, n)
        env.reset()
        model = PPO("MlpPolicy", env, verbose=1, batch_size=16, n_steps=64)
        model.learn(total_timesteps=128, progress_bar=False)
        acc = env.calculate_accuracy(env.action_idxes)
        accuracies_rl[n] = acc
        print('RL:', acc)
        
    df = pd.DataFrame(np.array(num_feats_to_test).T, columns=['num_feats'])
    accuracies_rfe_list = []
    accuracies_rl_list = []
    for feat in num_feats_to_test:
        accuracies_rfe_list.append(accuracies_rfe[feat])
        accuracies_rl_list.append(accuracies_rl[feat])
    df.loc[:, 'RFE Fused Model'] = np.array(accuracies_rfe_list).T
    df.loc[:, 'RL Fused Model'] = np.array(accuracies_rl_list).T

    fusion_plot_folder = os.path.join(args.output_path, 'fusion_plots', 'fused_model_accuracy')
    os.makedirs(fusion_plot_folder, exist_ok=True)
        
    # Plot accuracy for all features
    df.plot(x='num_feats')
    plt.title("Accuracy of Fused Models vs Number of Features")
    plt.xlabel("Number of Features")
    plt.ylabel("Accuracy")
    plt.savefig(f'{fusion_plot_folder}/fused_accuracies_2tl_all.png')
        
    # Plot accuracy up to 64 features
    df.plot(x='num_feats')
    plt.xlim([0, 64])
    plt.title("Accuracy of Fused Models vs Number of Features")
    plt.xlabel("Number of Features")
    plt.ylabel("Accuracy")
    plt.savefig(f'{fusion_plot_folder}/fused_accuracies_2tl_64.png')

def get_feature_importance(data, labels):

    fusion_plot_folder = os.path.join(args.output_path, 'fusion_plots', 'feature_importances')
    os.makedirs(fusion_plot_folder, exist_ok=True)

    model = DecisionTreeClassifier()
    model.fit(data, labels)
    importances = model.feature_importances_
    importances = np.array(importances)
    df = pd.DataFrame(importances.T, columns=['Feature Importance'])
        
    team_nums = []
    for sensor in range(1, args.num_sensors+1):
        team_nums.extend([f'Team {sensor}'] * NUM_TEAM_FEATURES[sensor])

    df.loc[:, 'Team'] = team_nums
        
    df = df.sort_values(['Feature Importance'], ascending=[False])
    df = df.reset_index()
    df = df.rename(columns={'index':'Feature Number'})

    # Plot each team's top 16 features
    color_bands = ['red', 'green', 'blue', 'yellow']
    for sensor in range(1, args.num_sensors+1):
        team_label = f'Team {sensor}'
        df_team = df.loc[df['Team'] == team_label]
        df_team = df_team.rename(columns={'Feature Importance': team_label})
        df_team.plot.bar(x='Feature Number', color=color_bands[sensor-1])
        plt.ylabel('Feature Importance')
        plt.title(f'Most Important Features, Team {sensor} Model')
        plt.xlim([0, 16])
        plt.show()
        plt.savefig(f'{fusion_plot_folder}/team{sensor}_2tl_feature_importance.png')

    # Plot the top 64 features from all models
    # NOTE: I couldn't figure out how to color bars based on which team's model they came from
    # With the current implementation, the bars need to be manually recolored, referencing the number of features extracted from each team's model
    # For example, with 65 feats from team 1, 512 from team 2, 64 from team 3, and 256 from team 4 (total of 897 feats), the breakdown is as follows:
    # Team 1: Feats 0 through 64
    # Team 2: Feats 65 through 576
    # Team 3: Feats 577 through 640
    # Team 4: Feats 641 through 896
    
    # Linear y axis
    figure(figsize=(10, 6))
    ax = plt.subplot(111)
    df.plot.bar(x='Feature Number', color='red', ax=ax)
    plt.xlim([0, 64])
    ax.tick_params(axis='x', which='major', labelsize=8)
    ax.tick_params(axis='x', which='minor', labelsize=6)
    custom_lines = [Line2D([0], [0], color='red', lw=4),
                    Line2D([0], [0], color='green', lw=4),
                    Line2D([0], [0], color='blue', lw=4),
                    Line2D([0], [0], color='yellow', lw=4)]
    ax.legend(custom_lines, ['Team 1', 'Team 2', 'Team 3', 'Team 4'])
    plt.ylabel('Feature Importance')
    plt.title('Most Important Features, All Models')
    plt.yscale('linear')
    plt.show()
    plt.savefig(f'{fusion_plot_folder}/all_models_2tl_feature_importance_lin.png')

    # Logarithmic y axis
    ax = plt.subplot(111)
    #figure(figsize=(10, 6))
    df.plot.bar(x='Feature Number', color='red', ax=ax)
    plt.xlim([0, 64])
    ax.tick_params(axis='x', which='major', labelsize=8)
    ax.tick_params(axis='x', which='minor', labelsize=6)
    custom_lines = [Line2D([0], [0], color='red', lw=4),
                    Line2D([0], [0], color='green', lw=4),
                    Line2D([0], [0], color='blue', lw=4),
                    Line2D([0], [0], color='yellow', lw=4)]
    ax.legend(custom_lines, ['Team 1', 'Team 2', 'Team 3', 'Team 4'])
    plt.ylabel('Feature Importance')
    plt.title('Most Important Features, All Models')
    plt.yscale('log')
    plt.show()
    plt.savefig(f'{fusion_plot_folder}/all_models_2tl_feature_importance_log.png')

if __name__ == "__main__":
    
    args, _ = parse_args()
    
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print('Using device', device)
    
    data, labels = load_data()

    create_rl_fused_model(data, labels)

    create_rfe_fused_model(data, labels)
    
    get_model_accuracy(data, labels)

    get_feature_importance(data, labels)

### Local Testing of File
Use the cell below to perform local testing of the file before launching a larger job on SageMaker. Make sure to update the file paths and args depending on the sample data in your local file system

In [7]:
!python ../code/rl_rfe_fusion.py --num-sensors 4 --input-path "/root/ClouddRF_Final/cloudd-rf/output/fusion_data" --output-path "/root/ClouddRF_Final/cloudd-rf/output"

Using device cuda
Fitting estimator with 897 features.
Fitting estimator with 797 features.
Fitting estimator with 697 features.
Fitting estimator with 597 features.
Fitting estimator with 497 features.
Fitting estimator with 397 features.
Fitting estimator with 297 features.
Fitting estimator with 197 features.
Fitting estimator with 97 features.
RFE Accuracy: 0.0
RFE: 0.0
(20, 897)
Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
[3 5 1 3 3 0 6 6 3 2 0 0 0 5 2 5 5 0 3 2]
[3 5 1 3 3 0 6 6 3 2 0 0 0 5 2 5 5 0 3 2]
[3 5 1 3 3 0 6 6 3 2 0 0 0 5 2 5 5 0 3 2]
[2 7 3 5 5 2 5 6 2 7 4 0 6 2 5 2 6 0 5 3]
[2 7 7 5 5 0 5 6 2 7 4 0 6 0 5 2 6 0 5 7]
[2 3 5 5 5 0 7 3 2 7 4 0 5 6 4 0 6 0 5 5]
[2 3 3 5 3 0 6 4 2 5 4 0 5 3 7 0 0 0 7 5]
[2 7 3 5 3 0 5 4 2 5 4 0 5 0 5 0 0 0 3 5]
[2 2 5 5 2 0 5 1 3 3 4 0 5 3 7 0 5 0 5 5]
[2 2 5 5 2 0 5 1 3 3 4 0 5 3 7 0 5 0 5 5]
[3 7 5 5 7 0 5 1 3 3 4 0 5 3 7 0 5 0 5 5]
[3 7 5 5 1 0 5 4 3 3 5 0 5 3 7 1 5 0 5 5]
[3 7 5 5 1 0 5

## Configure Processing Job

In [4]:
from sagemaker.pytorch.processing import PyTorchProcessor

from sagemaker.processing import (
    ProcessingInput,
    ProcessingOutput,
)
import time 

timestamp = str(time.time()).split('.')[0]
output_prefix = f'{base_job_prefix}/fusion/rl_rfe/outputs/{timestamp}'
output_s3_uri = f's3://{default_bucket}/{output_prefix}'
code_location = f's3://{default_bucket}/{base_job_prefix}/fusion/rl_rfe/code'

# S3 Location of Validation Dataset
# UPDATE the var below with the s3 prefix (just the portion after /preprocess) of where the validation data is located
s3_validation_data = f's3://{default_bucket}/{base_job_prefix}/evaluation/outputs/1730146700/fusion_data'

processing_instance_type = "ml.g4dn.xlarge"
processing_instance_count = 1
env_vars = {
    "SM_CHANNEL_VAL": "/opt/ml/processing/input/data/validation",
    "SM_OUTPUT_DIR": "/opt/ml/processing/output"
}

pytorch_processor = PyTorchProcessor(
    framework_version='1.13.1',
    py_version="py39",
    role=role,
    env=env_vars,
    instance_count=processing_instance_count,
    instance_type=processing_instance_type,
    base_job_name = f"{base_job_prefix}-fusion-rl-rfe",
    code_location=code_location
)

In [5]:
# Processing Script Arguments
num_sensors = 4 # Number of teams with distinct models

arguments = [
    "--num-sensors", str(num_sensors)
]

code = 'rl_rfe_fusion.py'

In [6]:
pytorch_processor.run(
                        code=code,
                        source_dir='../code',
                        arguments=arguments,
                        inputs=[
                            ProcessingInput(source=s3_validation_data, destination=env_vars["SM_CHANNEL_VAL"], s3_data_type='S3Prefix', s3_input_mode='File')
                       ],
                        outputs=[
                            ProcessingOutput(source=env_vars["SM_OUTPUT_DIR"], destination = output_s3_uri)
                        ]
                    )

INFO:sagemaker:Creating processing-job with name cloudd-rf-fusion-rl-rfe-2024-10-29-12-29-16-562


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 42.9 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 85.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.5/821.5 kB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.1/297.1 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 66.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.2/15.2 MB 87.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.3/182.3 kB 20.4 MB/s eta 0:00:00
Using cached Farama_Notifica

# Review Outputs

In [ ]:
s3_client = boto3.client("s3")
response = s3_client.list_objects_v2(Bucket=default_bucket, Prefix=output_s3_uri)
files = response.get("Contents")

for file in files:
    print(f"file_name: {file['Key']}, size: {file['Size']}")